In [1]:
import numpy as np
import torch
from torch import nn
import torch.autograd.functional as F

In [2]:
class Dynamics(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(Dynamics, self).__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.ReLU()
        )
    def forward(self, input_element):
        output = self.linear_relu_stack(input_element)
        return output

In [3]:
class reward(nn.Module):

    def __init__(self, input_size, hidden_size, output_size = 1):
        super(reward, self).__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size),
            nn.Tanh()
        )

    def forward(self, input_element):
        output = self.linear_relu_stack(input_element)
        return output

In [4]:
statelen = 10
actionlen = 3
learning_rate = 0.01

my_Dyna = Dynamics(statelen + actionlen, statelen, statelen)
my_reward = reward(statelen + actionlen, statelen , 1)

optimizer1 = torch.optim.SGD(my_Dyna.parameters(), lr=learning_rate)
optimizer2 = torch.optim.SGD(my_reward.parameters(), lr=learning_rate)

In [6]:

def forward_step(start_state, actions, time_step):
    state = start_state
    state_collection = start_state.unsqueeze(0)
    i = 0
    while i < time_step:
        sa_input = torch.cat((state,actions[i]),dim = 1)
        state = my_Dyna(sa_input)
        reward = my_reward(sa_input)
        
        if i == 0:
            reward_collection = reward.unsqueeze(0)
        else:
            reward_collection = torch.cat((reward_collection, reward.unsqueeze(0)), dim = 0)
            
        if i == time_step - 1:
            break
        state_collection = torch.cat((state_collection, state.unsqueeze(0)), dim = 0)
        
        i = i + 1
    
    return state_collection, reward_collection



In [7]:
start_s = torch.rand((1,10))
act_col = torch.rand((3,1,3))
state_col, reward_col = forward_step(start_s, act_col, 3)

In [6]:

T = 3
i = T

while i > 0:
    next_state = my_Dyna(sa_input)
    
    i = i - 1

next_state = torch.rand((1,10))
reward = torch.rand((1,1))

optimizer1.zero_grad()
optimizer2.zero_grad()

next_state_ = my_Dyna(testinput)
reward_ = my_reward(testinput)

loss1 = new_rew - testoutput2

loss2 = torch.sum(newstate - testoutput1)
loss = loss1 + loss2

loss.backward()

optimizer1.step()
optimizer2.step()

print(np.shape(new_rew))

torch.Size([1, 1])


In [7]:
C = F.hessian(rew, testinput.view(-1))

In [8]:
np.shape(C)

torch.Size([13, 13])

In [9]:
C_pre1 = torch.split(C,[10,3])[0]
C_pre2 = torch.split(C,[10,3])[1]

In [10]:
C_xx = torch.split(C_pre1,[10,3],dim = 1)[0]
C_xu = torch.split(C_pre1,[10,3],dim = 1)[1]
C_uu = torch.split(C_pre2,[10,3],dim = 1)[1]

print(np.shape(C_xx))
print(np.shape(C_xu))
print(np.shape(C_uu))

torch.Size([10, 10])
torch.Size([10, 3])
torch.Size([3, 3])


In [11]:
c_xtut = torch.matmul(testinput,C)
c_xtut = torch.matmul(c_xtut,torch.transpose(testinput, 0, 1))
print(np.shape(c_xtut))

torch.Size([1, 1])


In [12]:
print(np.shape(c_xtut))

torch.Size([1, 1])


In [13]:
np.shape(F.jacobian(rew, testinput.view(-1)))

torch.Size([1, 13])

In [14]:
reward_grad = F.jacobian(rew, testinput.view(-1))
xtut = torch.matmul(reward_grad, torch.transpose(testinput, 0, 1))

In [15]:
c_xtut = c_xtut*(1/2) + xtut

In [29]:
invCuu = torch.linalg.inv(C_uu)
print(invCuu)

tensor([[ 264.8106, -232.2623,   37.7186],
        [-232.2623,   21.1679,  -53.8926],
        [  37.7186,  -53.8926,  197.7308]])


In [30]:
K_t = -torch.matmul(invCuu, torch.transpose(C_xu,0,1))
print(K_t)

tensor([[-0.6638, -2.7066,  2.5375, -0.9293, -2.6375, -0.6081, -1.0358,  1.2644,
         -0.9148,  0.5197],
        [-0.6278,  0.0785, -0.6501, -0.5755,  0.6554,  0.0083, -0.7779,  0.7553,
         -0.5766,  0.5486],
        [-0.5171, -0.7170,  0.5261,  0.5118, -0.5461, -0.7213, -1.0095, -0.0376,
         -0.6958, -0.2683]])


In [23]:
np.shape(reward_grad)

torch.Size([1, 13])

In [24]:
c_t = torch.split(reward_grad.view(-1), [10, 3])

In [44]:
k_t = torch.matmul(invCuu, c_t[1])

tensor([-7.2472, -0.2899, -2.1184])


In [68]:
u_t = torch.matmul(K_t, torch.transpose(testinput,0,1)[0:10])

In [62]:
u_t = torch.transpose(u_t,0,1)

In [64]:
u_t = u_t + k_t

In [50]:
#for param in rew.parameters():
#    print(param)

In [52]:
np.shape(F.jacobian(Dyna, testinput))


torch.Size([10, 13])